In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.messages import SystemMessage, HumanMessage, AIMessage,ToolMessage



messages=[
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France? and tell me a joke about it")
]

model=ChatGroq(
       model="qwen/qwen3-32b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)
model.invoke(messages)

/Users/naveen/Desktop/web/genAi/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='The capital of France is **Paris**!  \n\nHere\'s a joke about it:  \n**Why did the Eiffel Tower get promoted?**  \n*Because it was the tallest in the office!*  \n\n😄 (And yes, Paris is also known as the "City of Light" and the "City of Love" — but the Eiffel Tower definitely knows how to climb the corporate ladder!)', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the capital of France and a joke about it. Let me start with the first part. The capital of France is Paris. That\'s straightforward.\n\nNow, for the joke. I need to come up with a joke that\'s related to Paris or France. Maybe something about the Eiffel Tower? Or maybe a play on words with "Paris" itself. Let me think... How about a pun? Like, "Why did the Eiffel Tower get cold? Because it was too Paris!" Wait, that\'s a bit forced. Maybe something else. \n\nAnother angle: Maybe a joke involving a French person or a stereotype. Like, "What do you call a fake noodle? An impasta." But 

In [ ]:
human_msg=HumanMessage(content="What is the capital of France? and tell me a joke about it",name="user")